In [7]:
import pandas as pd
import re


In [13]:
instructions = {
    'ADD': '18',
    'AND': '40',
    'COMP': '28',
    'DIV': '24',
    'J': '3C',
    'JEQ': '30',
    'JGT': '34',
    'JLT': '38',
    'JSUB': '48',
    'LDA': '00',
    'LDCH': '50',
    'LDL': '08',
    'LDX': '04',
    'MUL': '20',
    'OR': '44',
    'RD': 'D8',
    'RSUB': '4C',
    'STA': '0C',
    'STCH': '54',
    'STL': '14',
    'STSW': 'E8',
    'STX': '10',
    'SUB': '1C',
    'TD': 'E0',
    'TIX': '2C',
    'WD': 'DC',
    'FIX': ['C4', 1],
    'FLOAT': ['CO', 1],
    'HIO': ['F4', 1],
    'NORM': ['C8', 1],
    'SIO': ['F0', 1],
    'TIO': ['F8', 1],
}


In [14]:
def open_file(file):
    with open(file) as f:
        program = [re.sub(r'\s+', ' ', line).strip().upper() for line in f]
    return program

In [15]:
def instruction_format(instruction):
    res = 1 if type(instructions[instruction]) == list else  3
    return res

In [44]:
def return_df(program):
    dict = []
    for line in program:
        
        temp = line.split(" ")[1: 4]
        if len(temp) < 1:
            continue
        if temp[0] == '.':
            continue
            
        if temp[0] in instructions:
            temp = temp[: 2] 
        
        if temp[0]  == 'RSUB':
            label = ' '
            mnemonic = temp[0]
            value = ' '
            
        elif temp[0]  == 'END':
            label = ' '
            mnemonic = temp[0]
            value = temp[1]

        elif temp[1] == 'RSUB':
            label = temp[0]
            mnemonic = temp[1]
            value = ' '
        
            
        elif temp[0] in instructions:
            label = ' '
            mnemonic = temp[0]
            value = temp[1]

        elif len(temp) >= 3:
            label = temp[0]
            mnemonic = temp[1]
            value = temp[2]
        

        dict.append({
            'Label':label,
            'Mnemonic': mnemonic,
            'Value': value
        })
    df = pd.DataFrame(dict)

    if df.iloc[df.index.stop -1].Mnemonic != 'END':
        df2 = pd.DataFrame([[' ', 'END', df.iloc[0].Value]], columns=df.columns)
        df = pd.concat([df, df2], ignore_index = True)
    return df

In [46]:
program = open_file('in2.txt')
df = return_df(program)
df.to_csv('example.text', sep="\t", index = False, header = False)
df

,Label,Mnemonic,Value
0,PROG3,START,0030
1,WRREC,LDX,ZERO
2,WLOOP,TD,OUTPUT
3,,JEQ,WLOOP
4,,LDCH,"RECORD,X"
5,,WD,OUTPUT
6,,TIX,LENGTH
7,,TIX,LENGTH
8,ZERO,WORD,0
9,LENGTH,WORD,1


In [47]:
def prog_name(df):
    return df.iloc[0].Label.ljust(6, 'x')

In [48]:
name = prog_name(df)
name

'PROG3x'

In [49]:
fout = open("zeee.txt", "w", encoding='utf-8')
for ind in df.index:
    if(df.Label[ind] == ' '):
        fout.write('\t\t{0}\t{1}\n'.format(df.Mnemonic[ind].ljust(8, ' '), df.Value[ind]).ljust(8, ' '))
        
    else:
        fout.write('{0}\t{1}\t{2}\n'.format(df.Label[ind].ljust(8, ' '), df.Mnemonic[ind].ljust(8, ' '), df.Value[ind].ljust(8, ' ')))
fout.close()

In [50]:
df.index.stop

13

In [51]:
for i in range(1, df.index.stop):
    print(df.Label[i] + "\t", df.Mnemonic[i] + "\t", df.Value[i])

WRREC	 LDX	 ZERO
WLOOP	 TD	 OUTPUT
 	 JEQ	 WLOOP
 	 LDCH	 RECORD,X
 	 WD	 OUTPUT
 	 TIX	 LENGTH
 	 TIX	 LENGTH
ZERO	 WORD	 0
LENGTH	 WORD	 1
OUPUT	 BYTE	 X'05'
RECORD	 RESB	 100
 	 END	 0030


In [52]:
for ind in df.index:
    break
    #print(df.Label[ind] + "\t", df.Mnemonic[ind] + "\t", df.Value[ind])

In [53]:
df


,Label,Mnemonic,Value
0,PROG3,START,0030
1,WRREC,LDX,ZERO
2,WLOOP,TD,OUTPUT
3,,JEQ,WLOOP
4,,LDCH,"RECORD,X"
5,,WD,OUTPUT
6,,TIX,LENGTH
7,,TIX,LENGTH
8,ZERO,WORD,0
9,LENGTH,WORD,1


In [54]:
def location_counter(df):
    start_index = df.Value[0]
    list_counter = [" ", start_index]
    counter = start_index
    for i in range(1, df.index.stop):

        if df.Mnemonic[i] in instructions:
            if type(instructions[df.Mnemonic[i]]) == list:
                temp = hex(int(counter, 16) + 1)
            else:
                temp = hex(int(counter, 16) + 3)
            
        if df.Mnemonic[i] == 'WORD':
            temp = hex(int(counter, 16) + 3)
        if df.Mnemonic[i] == 'BYTE':
            value = df.Value[i].split('\'')
            length = len(value[1])
            if value[0] == 'C':
                temp = hex(int(counter, 16) + int(length))
            else:
                temp = hex(int(counter, 16) + int(int(length)/2))
                
        if df.Mnemonic[i] == 'RESW':
            temp = hex(int(counter, 16) + int(df.Value[i]) *3)
            
        if df.Mnemonic[i] == 'RESB':
            temp = hex(int(counter, 16) + int(df.Value[i]))
            
    
        counter = temp.split('x')[1].rjust(4, '0').upper()
        list_counter.append(counter)
        
    df.insert(0, 'Location_Counter', list_counter[:-1])
    return df

In [55]:
location_counter(df)


,Location_Counter,Label,Mnemonic,Value
0,,PROG3,START,0030
1,0030,WRREC,LDX,ZERO
2,0033,WLOOP,TD,OUTPUT
3,0036,,JEQ,WLOOP
4,0039,,LDCH,"RECORD,X"
5,003C,,WD,OUTPUT
6,003F,,TIX,LENGTH
7,0042,,TIX,LENGTH
8,0045,ZERO,WORD,0
9,0048,LENGTH,WORD,1


In [56]:
sum = hex(int('1000', 16) + 3)
sum[2: ]

'1003'

In [63]:
def symbol_table(df):
    list = []
    for ind in range(1, df.index.stop):
        if df.Label[ind] != ' ':
            list.append([df.Label[ind], df.Location_Counter[ind]])
    return list

In [64]:
symbol_table(df)

[['WRREC', '0030'],
 ['WLOOP', '0033'],
 ['ZERO', '0045'],
 ['LENGTH', '0048'],
 ['OUPUT', '004B'],
 ['RECORD', '004C']]